# Introduction

This notebook will train a very simple model that will compare two numbers and tell us if one is larger than the other.

We'll convert the model into something that can be used by tflite and then run it on the ESP32

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from tensorflow.data import Dataset
import numpy as np
import itertools
from tensorflow.keras import backend as K

2022-04-22 11:18:45.702650: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 11:18:45.702696: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Create a dataset to train our model
We'll create a python generator and feed that through a tensorflow Dataset to train our model

In [2]:
def data_generator():
    while(True):
        number1 = np.random.uniform();
        number2 = np.random.uniform();
        # our input data is an array containing 2 numbers
        X = [number1, number2]
        # our label is 1 or 0
        Y = 1 if number2 > number1 else 0
        # our generator should return the input data and the label
        yield X, [Y]
        
# create a dataset from our generator
train_dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_types = (tf.float32, tf.int32),
    output_shapes=((2), (1))
)
train_dataset = train_dataset.batch(batch_size=30)

2022-04-22 10:01:10.532594: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-22 10:01:10.532676: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-22 10:01:10.532706: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vmtg-linux): /proc/driver/nvidia/version does not exist
2022-04-22 10:01:10.534172: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Our very simple mode

We don't need a very complicated model for our problem, so we'll just define a small neural network with an input layer and an output layer.

It's important that the activation function for the output should be sigmoid. This activation function will output a value between 0 and 1.

In [3]:
model = Sequential([
    Input(shape=(2) #Shape defines the length of the input vector 
    ),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')
])

## Compile our model¶
For our loss function we need to use BinaryCrossentropy.

Crossentropy quantifies the difference between two probability distribution.

We have a binary distribution (True or False) so we use binary crossentropy to compare the output from our model with the true distribution.


In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 15        
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


The first layer has 15 parameters: 
Each neuron has values from the vector (2 values * 5 Neurons = 10) and the biases 

In [5]:
model.fit(
    train_dataset,
    steps_per_epoch=1000,
    epochs=4
)

Epoch 1/4
1000/1000 [==============================] - 11s 9ms/step - loss: 0.5052 - accuracy: 0.8328
Epoch 2/4
1000/1000 [==============================] - 9s 9ms/step - loss: 0.2783 - accuracy: 0.9658
Epoch 3/4
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1435 - accuracy: 0.9943
Epoch 4/4
1000/1000 [==============================] - 9s 9ms/step - loss: 0.1059 - accuracy: 0.9961


## Testing our model
We can feed in some values and see what our model predicts

In [14]:
test_X = np.array([
    [0.1, 0.0],
    [0.3, 0.1],
    [0.5, 0.1],
    [0.7, 0.2]
])
Y = model.predict_on_batch(test_X)
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})
print(Y)

[[0.15]
 [0.03]
 [0.01]
 [0.00]]


## Extracting the weights of the network

In [16]:
model.get_weights()

[array([[-1.51, -2.14, 2.22, -0.58, -0.46],
        [1.88, 2.40, -1.17, 0.04, -0.60]], dtype=float32),
 array([0.31, 0.53, 0.74, -0.04, 0.00], dtype=float32),
 array([[3.12],
        [3.33],
        [-2.11],
        [-0.38],
        [-0.33]], dtype=float32),
 array([-1.23], dtype=float32)]

## Extracting the output of the first layer

In [12]:
list(map(lambda x: x.name, model.layers))

['dense', 'dense_1']

In [7]:
"""Keras function for getting the output of a given layer
This function is created acording to the layer_indexthe function works in the following way
Input: entry data of the neuron 
Output: output of the network  
The function below works for the input neuron  """
layer_index = 0
output_function = K.function([model.layers[0].input], model.get_layer(index=layer_index).output)

In [8]:
#Testing the function with the previous data 
layer_output = output_function([test_X])  # input_data is a numpy array
print(layer_output)

[[0.53 0.80 0.72 0.00 0.00]
 [0.61 0.85 0.93 0.00 0.00]
 [0.00 0.00 1.73 0.00 0.00]
 [0.00 0.00 2.06 0.00 0.00]]
